<a href="https://colab.research.google.com/github/TahniyatJavaid30/stock-trend-predictor/blob/main/tahniyat_javaid_project_1%2C_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install the required Python libraries for financial data analysis and forecasting**

In [29]:
!pip install yfinance pandas numpy scikit-learn prophet plotly

**Import necessary libraries**

In [30]:
import yfinance as yf              # For downloading financial (stock market) data from Yahoo Finance
import pandas as pd
import plotly.graph_objects as go  # For creating interactive and advanced visualizations (like stock charts, line charts, etc.)
from prophet import Prophet        # For time series forecasting (predicting future stock prices or trends)


**Fetch and Display Stock Data from Yahoo Finance**

In [44]:
# Define a function to get stock data from Yahoo Finance
def fetch_stock_data(ticker='AAPL', period='6mo', interval='1d'):
    # Download stock data for the given company
    stock = yf.download(ticker, period=period, interval=interval)

    # Reset the index so that the date becomes a normal column
    stock.reset_index(inplace=True)

    # Fix column names in case they come as tuples
    stock.columns = [col[0] if isinstance(col, tuple) else col for col in stock.columns]

    # Return the final cleaned data
    return stock

# Run the function to get Apple stock data
df = fetch_stock_data('AAPL')

# Print a message to confirm that data was downloaded
print("Data fetched successfully!\n")

# Show the first 10 rows of the dataset
print(df.head(10))


/tmp/ipython-input-1615789562.py:4: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed

Data fetched successfully!

        Date       Close        High         Low        Open     Volume
0 2025-04-25  208.768982  209.237836  205.696501  205.866084   38222300
1 2025-04-28  209.626892  210.983572  206.953443  209.487235   38743100
2 2025-04-29  210.694275  211.721759  207.861198  208.180424   36827600
3 2025-04-30  211.981125  213.058490  206.165359  208.788942   52286500
4 2025-05-01  212.799133  214.036096  208.389912  208.569481   57365700
5 2025-05-02  204.848587  206.484582  201.666374  205.586770  101010600
6 2025-05-05  198.404358  203.601643  197.726026  202.604085   69018500
7 2025-05-06  198.025269  200.160042  196.538917  197.726013   51216500
8 2025-05-07  195.770798  198.953011  192.778123  198.683666   68536700
9 2025-05-08  197.007782  199.561529  194.204631  197.237216   50478900


**Clean and Prepare Stock Data**

In [47]:
# Define a function to process and clean the stock data
def process_stock_data(df):
    # Calculate 10-day moving average of the closing price
    df['MA10'] = df['Close'].rolling(window=10).mean()

    # Calculate 30-day moving average of the closing price
    df['MA30'] = df['Close'].rolling(window=30).mean()

    # Calculate daily return (percentage change in closing price)
    df['Daily Return'] = df['Close'].pct_change()

    # Convert the 'Date' column to proper date format
    df['Date'] = pd.to_datetime(df['Date'])

    # Make sure 'Close' column has numeric values only
    df['Close'] = pd.to_numeric(df['Close'], errors='coerce')

    # Remove rows where 'Close' value is missing (NaN)
    df.dropna(subset=['Close'], inplace=True)

    # Return the cleaned and processed data
    return df


**Clean Stock Data and Display Sample Records**

In [33]:
# Run the function to clean and process the stock data
df = process_stock_data(df)

# Print a message to confirm that data cleaning was successful
print(" Data cleaned successfully!\n")

# Show the first 10 rows of the cleaned data
print(df.head(10))


 Data cleaned successfully!

        Date       Close        High         Low        Open     Volume  \
0 2025-04-25  208.768982  209.237836  205.696501  205.866084   38222300   
1 2025-04-28  209.626892  210.983572  206.953443  209.487235   38743100   
2 2025-04-29  210.694275  211.721759  207.861198  208.180424   36827600   
3 2025-04-30  211.981125  213.058490  206.165359  208.788942   52286500   
4 2025-05-01  212.799133  214.036096  208.389912  208.569481   57365700   
5 2025-05-02  204.848587  206.484582  201.666374  205.586770  101010600   
6 2025-05-05  198.404358  203.601643  197.726026  202.604085   69018500   
7 2025-05-06  198.025269  200.160042  196.538917  197.726013   51216500   
8 2025-05-07  195.770798  198.953011  192.778123  198.683666   68536700   
9 2025-05-08  197.007782  199.561529  194.204631  197.237216   50478900   

        MA10  MA30  Daily Return  
0        NaN   NaN           NaN  
1        NaN   NaN      0.004109  
2        NaN   NaN      0.005092  
3    

**Statistical Summary of the Dataset**

In [34]:
# Show basic statistical summary of the data
df.describe()

,Date,Close,High,Low,Open,Volume,MA10,MA30,Daily Return
count,127,127.000000,127.000000,127.000000,127.000000,1.270000e+02,118.000000,98.000000,126.000000
mean,2025-07-26 10:34:57.637795328,221.800267,223.869215,219.584083,221.667773,5.377413e+07,220.912104,219.292209,0.001952
min,2025-04-25 00:00:00,195.048645,197.346030,192.778123,193.450453,3.062120e+07,198.717473,200.996887,-0.037362
25%,2025-06-10 12:00:00,203.079529,206.644934,201.558873,204.218234,4.230000e+07,204.352925,204.029158,-0.005786
50%,2025-07-28 00:00:00,212.799133,214.606454,210.581016,212.189172,4.806810e+07,211.400079,214.825903,0.000922
75%,2025-09-10 12:00:00,237.880005,239.375000,235.675003,237.195000,5.682710e+07,234.670751,233.454749,0.006332
max,2025-10-24 00:00:00,262.820007,265.290009,261.829987,262.649994,1.637413e+08,256.322000,252.431332,0.063146
std,NaN,20.665100,20.588955,20.393196,20.461035,1.993206e+07,19.311393,16.528562,0.015848


**View Recent Stock Data with Moving Averages and Daily Returns**

In [35]:
# Show the last 10 rows of selected columns: Date, Close, MA10, MA30, and Daily Return
df[['Date', 'Close', 'MA10', 'MA30', 'Daily Return']].tail(10)


,Date,Close,MA10,MA30,Daily Return
117,2025-10-13,247.660004,254.343001,246.033666,0.009744
118,2025-10-14,247.770004,253.657001,246.635333,0.000444
119,2025-10-15,249.339996,253.046001,246.997666,0.006336
120,2025-10-16,247.449997,252.078000,247.253333,-0.007580
121,2025-10-17,252.289993,251.505000,247.673333,0.019559
122,2025-10-20,262.239990,252.059999,248.485332,0.039439
123,2025-10-21,262.769989,252.688997,249.432665,0.002021
124,2025-10-22,258.450012,252.727998,250.487999,-0.016440
125,2025-10-23,259.579987,253.281998,251.472999,0.004372
126,2025-10-24,262.820007,255.036998,252.431332,0.012482


**Predict Future Stock Prices Using Prophet**

In [50]:
# Import Prophet
from prophet import Prophet

# Define a function to predict future stock prices
def predict_trend(df):
    # Prepare the data for Prophet (rename columns to 'ds' for date and 'y' for value)
    prophet_df = df[['Date', 'Close']].rename(columns={'Date': 'ds', 'Close': 'y'})

    # Create a Prophet model and set it to work with daily data
    model = Prophet(daily_seasonality=True)

    # Train (fit) the model on our data
    model.fit(prophet_df)

    # Make a future dataframe for the next 7 days
    future = model.make_future_dataframe(periods=7)

    # Predict future stock prices
    forecast = model.predict(future)

    # Return the forecasted data
    return forecast

# Run the function to predict future trend
forecast = predict_trend(df)

# Show the last 7 forecasted values
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7))


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpct5zvups/nic83ffs.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpct5zvups/grg_ljrj.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=59887', 'data', 'file=/tmp/tmpct5zvups/nic83ffs.json', 'init=/tmp/tmpct5zvups/grg_ljrj.json', 'output', 'file=/tmp/tmpct5zvups/prophet_modelhd_ax3gr/prophet_model-20251027113036.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
11:30:36 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
11:30:37 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


            ds        yhat  yhat_lower  yhat_upper
127 2025-10-25  263.046474  256.005173  270.130106
128 2025-10-26  263.591301  256.011416  271.096345
129 2025-10-27  264.241785  257.205874  271.662980
130 2025-10-28  264.317430  256.792774  271.341919
131 2025-10-29  264.787187  257.564036  272.327393
132 2025-10-30  265.607008  258.467971  273.209383
133 2025-10-31  265.759543  258.231645  272.813362


**Visualize Actual vs Predicted Stock Prices**

In [38]:
# Create a new interactive figure
fig = go.Figure()

# Add actual stock price data to the graph
fig.add_trace(go.Scatter(x=df['Date'], y=df['Close'], name='Actual Price'))

# Add predicted stock price data to the graph
fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'], name='Predicted Price'))

# Add title and labels to the chart
fig.update_layout(
    title="📈 Real-Time Stock Market Trend Predictor",    # Chart title
    xaxis_title="Date",                                   # X-axis label
    yaxis_title="Price (USD)",                            # Y-axis label
    template="plotly_dark")                               # Dark theme for better visuals



**View Last 7 Days of Stock Price Forecast**

In [39]:
# Show the last 7 days of forecast data (predicted price, and its lower & upper limits)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7)


,ds,yhat,yhat_lower,yhat_upper
127,2025-10-25,263.046474,255.712463,270.782617
128,2025-10-26,263.591301,255.871134,271.290102
129,2025-10-27,264.241785,256.893902,271.768775
130,2025-10-28,264.317430,256.978264,271.693680
131,2025-10-29,264.787187,257.901384,272.752410
132,2025-10-30,265.607008,258.765248,273.145085
133,2025-10-31,265.759543,258.516905,273.148197


**Highlight Last 7 Days of Predicted Prices on Chart**

In [40]:
# Add the predicted prices for the last 7 days to the chart
fig.add_trace(go.Scatter(x=forecast.tail(7)['ds'], y=forecast.tail(7)['yhat'], name='Predicted Price'))